In [28]:
import pandas as pd
import openai
import json
import re 
openai.api_key = ""

json_pattern = re.compile(r'\{.*?\}', re.DOTALL)


In [ ]:
events = pd.read_csv('Terr.csv')
topics = pd.read_csv('Topics.csv')
output_file_path = 'image-prompts.txt'


In [ ]:
events_json = events.to_json(orient='records')
events_json = json.loads(events_json)


for i in range(0, len(events_json), 5):
    chunk = json.dumps(events_json[i:i + 5], indent=2)
    for index, row in topics.iterrows():
        output_json = {
        "EID": "",  
        "Topic": row.get('Topic', ''),
        "Prompt": ""}
        json_output = json.dumps(output_json, indent=4)
        
        prompt_text = f"""
        Using the row from the provided JSON FILE, which lists specific events, historical figures, and associated terminology, generate three detailed large prompts for creating images related to the theme of "{row['Topic']}". Each prompt should visualize real-world scenes associated with negative things like warfare, conflict, or socio-political tension, focusing on realistic settings, equipment, and environments. The prompts must be long to capture every detail about scene and ensure it is real life. Emphasize elements such as equipment, visual deception, propaganda, and the complexities of conflict environments to highlight their impact on public perception and emotions. The visual narrative should be compelling and research-oriented, aiming to describe {row['Description']}.
        Input -
        {chunk}


        The output should be a json in this format
        {json_output}

        Each prompt must be distinct, focusing on different aspects and scenes related to the theme.

        """
        
        messages = [
        {"role": "system", "content": "You are an AI assistant with expertise in creative writing and generating image prompts based on detailed scenarios."},
        {"role": "user", "content": f"""{prompt_text}
        Output - 
        """}]

        response = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages
        )

        assistant_response = response['choices'][0]['message']['content']

        match = json_pattern.search(assistant_response)
        if match:
                json_string = match.group(0)  # Get the matched JSON string
                try:
                    extracted_json = json.loads(json_string) 
                except:
                    print("Error: The extracted string is not valid JSON.")

        with open(output_file_path, 'a') as file:
                    file.write(json.dumps(extracted_json, indent = 2) + '\n')

